<a href="https://colab.research.google.com/github/eugenerz/auto-limpieza-notebook/blob/main/Zocalito_Formato_de_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import urllib.parse
import sqlite3
from gspread_dataframe import set_with_dataframe
import numpy as np
import re
import ast
import sys
sys.path.append('/content/drive/MyDrive/Limpieza_de_datos')
import mi_libreria as ml
ml.autenticar_drive()
gc = ml.autenticar_google()
conn, cursor = ml.conectar_sqlite_memoria()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd

def obtener_nuevos_registros(df_nueva, df_limpia, columna_id='ID_registro'):
    """
    Filtra los registros nuevos que no están en la base limpia.

    Parámetros:
        df_nueva (pd.DataFrame): DataFrame con la base actualizada.
        df_limpia (pd.DataFrame): DataFrame con la base ya limpia.
        columna_id (str): Nombre de la columna de ID único.

    Retorna:
        pd.DataFrame: Subconjunto de df_nueva con solo los registros nuevos.
    """
    ids_limpios = set(df_limpia[columna_id])
    nuevos_registros = df_nueva[~df_nueva[columna_id].isin(ids_limpios)].copy()
    return nuevos_registros

## CARGADO DE BASES


In [125]:

## BASE DE RESPUESTAS
sheet_id = "1PHXA-6UKh4XmHE6yy8zYvMiYAgw_D1u8iyyE4T6Tp3c"

sheet_name = "Respuestas"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
base_raw=pd.read_csv(url1, encoding='UTF8', dtype=str)







In [126]:

## BASE DE CODIGOS
sheet_id2 = "1m5DYExVVMKglqjFFMrsp1oDMXS_PkLKEJIKg_QMo6-M"

sheet_name2 = "Codigos"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name2}"
codigos=pd.read_csv(url1, encoding='UTF8', dtype=str)



In [127]:

## BASE DE CLASIFICACION
sheet_name3 = "Clasificacion"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3}"
clasificacion_abiertas=pd.read_csv(url1, encoding='UTF8', dtype=str)

## BASE LIMPIA

sheet_name3_1 = "Base Limpia"
sheet_name3_1_encoded = urllib.parse.quote(sheet_name3_1)


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3_1_encoded}"
base_limpia=pd.read_csv(url1, encoding='UTF8', dtype=str)

In [128]:
## Base de ID's


sheet_id4 = "1kb0JidVyZLStW7-S1jvjcs5WMXxMqyA_o_kvmSnhAvc"
sheet_name4 = "Hoja 1"

sheet_name4_encoded = urllib.parse.quote(sheet_name4)

url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id4}/gviz/tq?tqx=out:csv&sheet={sheet_name4_encoded}"
base_id_equipo=pd.read_csv(url1, encoding='UTF8', dtype=str)

diccionario_correo_a_id = base_id_equipo.set_index("CORREO")["ID"].to_dict()

## FORMATEO DE TITUTLOS Y IDS

In [129]:
base=base_raw.copy()

In [130]:
columnas=ml.extraer_headers(base)
base.columns=columnas["Codigo"]
base['index']=base.index
base.insert(0,'index',base.pop('index'))

In [131]:
base['Marca temporal'] = pd.to_datetime(base['Marca temporal'], errors='coerce')
base['Fecha'] = base['Marca temporal'].dt.date
base['Hora'] = base['Marca temporal'].dt.time

<ipython-input-131-d7552457d62b>:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  base['Marca temporal'] = pd.to_datetime(base['Marca temporal'], errors='coerce')


In [132]:
base.insert(1,'Fecha',base.pop('Fecha'))
base.insert(2,'Hora',base.pop('Hora'))
base['Hora'] = base['Hora'].astype(str).str[:5]

In [133]:
base["Marca temporal"] = pd.to_datetime(base["Marca temporal"], errors="coerce")
base["ID_registro"] = base["Marca temporal"].dt.strftime('%Y%m%d%H%M%S')
base.insert(1, 'ID_registro', base.pop('ID_registro'))

In [134]:
ml.validar_ids_unicos(base, "ID_registro")

✅ Todos los valores en la columna 'ID_registro' son únicos.


## Limpieza

In [135]:
base['Dirección de correo electrónico']=base['Dirección de correo electrónico'].map(diccionario_correo_a_id).fillna(base['Dirección de correo electrónico'])
base.rename(columns={'Dirección de correo electrónico': 'ID Encuestador'}, inplace=True)

In [136]:
codigos['Tipo de Pregunta'].value_counts()

,count
Tipo de Pregunta,
MULTIPLE,48
ABIERTA,10
INDICADORES,4
ABIERTA (NO CLASIFICABLE),1
DE REFERENCIA,1


In [137]:

# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['MULTIPLE', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
columnas_a_procesar = multiple_reference_rows['Codigo'].tolist()



In [138]:
## extraccion de codigos de repsuesta en multiples
for col in columnas_a_procesar:
    base[col] = base[col].apply(lambda x: ml.extraer_antes_del_punto(x))


In [139]:
base.replace("nan", "", inplace=True)
base.replace("Nan", "", inplace=True)
base.replace("NAN", "", inplace=True)
base.replace(np.nan, "", inplace=True)

In [140]:
base_updated=obtener_nuevos_registros(base, base_limpia, columna_id='ID_registro')

In [141]:
base_updated

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,ID Encuestador,H,COD,S1,D1,...,D4,P19,NSE 01,NSE 02,NSE 03,NSE 04,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
55,55,20250430105848,2025-04-30,10:58,2025-04-30 10:58:48,E16,10:49:00 a.m.,C1,SI,2,...,3,3,3,2,2,2,2,2,ACEPTO,
56,56,20250430105912,2025-04-30,10:59,2025-04-30 10:59:12,E34,10:54:00 a.m.,C1,SI,1,...,8,2,11,2,2,1,2,3,ACEPTO,
57,57,20250430110000,2025-04-30,11:00,2025-04-30 11:00:00,E38,10:55:00 a.m.,C4,SI,1,...,99,3,99,9,9,9,9,9,ACEPTO,
58,58,20250430110001,2025-04-30,11:00,2025-04-30 11:00:01,E23,10:53:00 a.m.,C5,SI,2,...,7,3,5,3,1,1,3,3,ACEPTO,
59,59,20250430110043,2025-04-30,11:00,2025-04-30 11:00:43,E2,10:52:00 a.m.,C5,SI,1,...,14,4,9,2,1,1,4,4,ACEPTO,
60,60,20250430110104,2025-04-30,11:01,2025-04-30 11:01:04,jimenezfaby115@gmail.com,10:53:00 a.m.,C5,SI,2,...,1,3,9,2,1,1,2,3,ACEPTO,
61,61,20250430110151,2025-04-30,11:01,2025-04-30 11:01:51,lorenamireyamelesioespitia@gmail.com,10:55:00 a.m.,C2,SI,2,...,7,3,9,3,2,1,5,5,ACEPTO,
62,62,20250430110209,2025-04-30,11:02,2025-04-30 11:02:09,fannyespindola7@gmail.com,10:55:00 a.m.,C2,SI,2,...,2,3,5,2,2,1,3,3,ACEPTO,


In [142]:
ml.agregar_dataframe_a_hoja(sheet_id2,base_updated,"Base Limpia",gc)

DataFrame agregado a partir de la fila 57 en la hoja 'Base Limpia'.


## EXTRACION DE PREGUNTAS ABIERTAS

In [143]:
try:
    del codigos_abiertas
except NameError:
    pass  # Ignore if the variable doesn't exist



In [144]:
# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['ABIERTA', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
codigos_abiertas = multiple_reference_rows['Codigo'].tolist()

In [145]:
codigos_abiertas.append('Fecha')
codigos_abiertas.append('Hora')
codigos_abiertas.append('index')
codigos_abiertas.append('ID_registro')

In [146]:
base_abiertas=base[codigos_abiertas]

In [147]:
base_abiertas

Codigo,P3_10_otro,P7_1,P7_2,P8,P9,P13,P14,P15,P16,P17,P18,Fecha,Hora,index,ID_registro
0,,Por qué sirve para distracción,Nada más,Ninguna,Juegos,Pinta carita,Ninguna,1,Para tener distracción,Ninguna,Albercas,2025-04-30,10:28,0,20250430102850
1,,Dibujo,Ns,Ns,Sibujo,Dinujo,Ns,1,De desapegan del cel,Ns,Ns,2025-04-30,10:29,1,20250430102901
2,99,DOMINO,GENGA,GENGA,99,DOMINO,GENGA,1,SE ENTRETIENEN,99,ALBERCAS,2025-04-30,10:33,2,20250430103315
3,99,Tatuajes,Linea de tiempo,Trampolines,Ninguna,Trampolines,99,1,Los saca de su sona de confor,Más organización,Manualidades para todo tipo de edades,2025-04-30,10:33,3,20250430103329
4,,Pinta caritas,Ns,Ns,Ns,Pinta caritas,Ns,1,Los motiva ha hacer diferentes actividades,Ns,El 10 de mayo,2025-04-30,10:33,4,20250430103334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,99,SERPIENTE Y ESCALERAS,GENGA,ARMAR FLORES,99,TODAS,NINGUNO,1,ES MEJOR QUE LOS CELULARES,QUE LO SIGAN ASIENDO,ACTIVIDADES PARA LOS DE LA TERCERA EDAD,2025-04-30,11:00,58,20250430110001
59,99,Inflabe,Libros,Los inflables,Todo bien,Todas,En ninguno,1,Tienen contacto con niños de su edad q estar s...,Ninguna todo bien,Taller de robótica,2025-04-30,11:00,59,20250430110043
60,,Pintura,Inflables,Pintura,Pintar,Inflables,Tirolesa,1,Por qué es bueno esto nunca hubo en la cdmx,Que sean más atentos los que están adentro,Los juegos para niños menores de 4 años,2025-04-30,11:01,60,20250430110104
61,,Pinta caritas,Futbol,Brincilin,Futbol,Inflable,99,1,Los ayuda en el desarrollo soco motriz,Mas organización y más personal,Bailes y juegos de mesa,2025-04-30,11:01,61,20250430110151


In [148]:
# prompt: añadir una columna vacia a lado de cada columna en base_abiertas cuyo Tipo de pregunta en codigos sea ABIERTA, el nombre de la columna debe ser del formato columna_clas

for col in base_abiertas.columns:
    tipo_pregunta = codigos[codigos['Codigo'] == col]['Tipo de Pregunta'].iloc[0] if col in codigos['Codigo'].values else None
    if tipo_pregunta == 'ABIERTA':
        nueva_col = col + '_clas'
        base_abiertas.insert(base_abiertas.columns.get_loc(col) + 1, nueva_col, '')


In [149]:
base_abiertas_updated=obtener_nuevos_registros(base_abiertas, clasificacion_abiertas, columna_id='ID_registro')

In [150]:
gc = ml.autenticar_google()
ml.agregar_dataframe_a_hoja(sheet_id2,base_abiertas_updated,"Clasificacion",gc)

DataFrame agregado a partir de la fila 2 en la hoja 'Clasificacion'.


### cargado de base a FORMATO



## EJECUCION DE ACTUALIZACION